<a href="https://colab.research.google.com/github/ElizabethEspinosa/County-Cover-Crop-Adoption-And-Disadoption/blob/main/Code/Cropswitching_specialization_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-emespinosa')

# **Import Cropland Data Layer from 2008 to 2023**


In [ ]:

collection = (
    ee.ImageCollection('USDA/NASS/CDL')
    .select('cropland').filterDate('2008-01-01', '2023-12-31'));

# Center map
m = geemap.Map(center=[39.8333, -98.5833], zoom=4)

# Add the image layer to the map and display it.
#m.add_layer(collection.first())
#display(m)

**Select pixels with cultivated crops**

In [ ]:
# Function to select appropiate codes (remove developed areas, clouds, water, other crops, hay, barren, etc. are nan)
def mask_cropland(img):
  mask= img.lte(61).Or(img.gt(65).And(img.lt(70))).Or(img.gt(71).And(img.lt(78))).Or(img.gte(204))
  return img.updateMask(mask)

#
cropland = collection.map(mask_cropland)

In [ ]:
# Function to select crop codes and remove other crops, alfalfa, and fallow cropland (1,0)
def mask_onlycrops(img):
  mask= img.lt(37).Or(img.gt(37).And(img.lt(44))).Or(img.gt(44).And(img.lt(59))).Or(img.gt(65).And(img.lt(70))).Or(img.gt(71).And(img.lt(78))).Or(img.gte(204))
  return ee.Image(mask)

onlycrops = collection.map(mask_onlycrops)

In [ ]:
#cropland.toBands().getInfo()


In [ ]:
# Function to replace repeated values from the analyzed crop

def blend_values(img, from_list, to_list):
  ## remap (replace crop values to one crop code)
  img_remap = img.remap(from_list, to_list) # default values are masked (0)
  ## blend with original-complete cropland image to fill masked values with cropland values
  #for i in list(range(15)): # total years of data

  blended = img_remap.blend(img)

  return blended

In [ ]:
## CORN collection map
corn_from = [225, 226, 228, 237, 241]
corn_to = [1, 1, 1, 1, 1]
corn = cropland.map(lambda img: blend_values(img, corn_from, corn_to))


In [ ]:
## SOY collection map
soy_from = [26, 239, 240, 241]
soy_to = [5,5,5,5]
soy = cropland.map(lambda img:blend_values(img, soy_from, soy_to))

## SORGHUM collection map
#sorg_from = [234, 235, 236]
#sorg_to = [4, 4, 4]
#sorg = cropland.map(lambda img:blend_values(img, sorg_from, sorg_to))

## COTTON collection map
cot_from = [232, 238, 239]
cot_to = [2, 2, 2]
cot = cropland.map(lambda img:blend_values(img, cot_from, cot_to))

## BARLEY collection map
#bar_from = [233, 235, 237, 254]
#bar_to = [21, 21, 21, 21]
#bar = cropland.map(lambda img:blend_values(img, bar_from, bar_to))

## WHEAT collection map
#wheat_from = [23, 24, 26, 39, 225, 230, 234, 236, 238]
#wheat_to = [22, 22, 22, 22, 22, 22, 22, 22, 22]
#wheat = cropland.map(lambda img:blend_values(img, wheat_from, wheat_to))

## RICE collection map
rice_from = [3]
rice_to = [3]
rice = cropland.map(lambda img:blend_values(img, rice_from, rice_to))

In [ ]:
def field_crops(img): #binary variable for commodity crops (i.e., corn, soybeans, sorghum, cotton, rice, wheat)
  mask= img.lt(6).Or(img.gte(22).And(img.lte(24))).Or(img.eq(26)).Or(img.eq(225)).Or(img.eq(226)).Or(img.eq(228)).Or(img.eq(230)).Or(img.eq(232)).Or(img.gte(234).And(img.lte(241))).Or(img.gte(254))
  return ee.Image(mask)

fieldcrops = collection.map(commodity_crops)

In [ ]:
# Convert collection of images of each year to bands in one image (to facilitate calculations)
main_img = cropland.toBands()
print('Collection to bands:', main_img.bandNames().getInfo()) #this is ok for RICE

mask_img = onlycrops.toBands()

corn_coll = corn.toBands()
soy_coll = soy.toBands()
#sorg_coll = sorg.toBands()
cot_coll = cot.toBands()
#bar_coll = bar.toBands()
#wheat_coll = wheat.toBands()
rice_coll= rice.toBands()
#commodity_coll= commcrops.toBands()
#print('commodity_coll:', rice_coll.bandNames().getInfo())


Collection to bands: ['2008_cropland', '2009_cropland', '2010_cropland', '2011_cropland', '2012_cropland', '2013_cropland', '2014_cropland', '2015_cropland', '2016_cropland', '2017_cropland', '2018_cropland', '2019_cropland', '2020_cropland', '2021_cropland', '2022_cropland', '2023_cropland']


##**Reference period characterization**

In [ ]:
n= 7
# Reference years from 2008 to 2014
### Remove fallow land from main image (if fallow land is present along the 6 years of the reference period)
listRef_cols= list(range(7)) # first 7 years
f= main_img.select(listRef_cols).eq(61)
fa= f.expression(
      '(A+B+C+D+E+F+G)',
      {'A': f.select(0),
        'B': f.select(1),
        'C': f.select(2),
        'D': f.select(3),
        'E': f.select(4),
        'F': f.select(5),
        'G': f.select(6)
        })
fal= fa.eq(n) ## all fallow during reference period= 1
fallow_img = ee.Image(fal).rename('fallow') # obtain one image that will be used later as filter


In [ ]:
c= fa.lt(7) ## all nonfallow during reference period= 1
nonfallow_img = ee.Image(c).rename('nonfallow')

###**Period 2**

In [ ]:
# Reference period 2 from 2013 to 2019
### Remove fallow land from main image (if fallow land is present along the 6 years of the reference period)
listRef_cols= list(range(5,12))
f= main_img.select(listRef_cols).eq(61)
fa= f.expression(
      '(A+B+C+D+E+F+G)',
      {'A': f.select(0),
        'B': f.select(1),
        'C': f.select(2),
        'D': f.select(3),
        'E': f.select(4),
        'F': f.select(5),
        'G': f.select(6)
        })
fal= fa.eq(n) ## all fallow during reference period= 1
fallow_img2 = ee.Image(fal).rename('fallow') # obtain one image that will be used later as filter


In [ ]:
c= fa.lt(7) ## all nonfallow during reference period= 1
nonfallow_img2 = ee.Image(c).rename('nonfallow')

### **Identify specialized crop sequences**

In [ ]:
## A Specialized crop sequence is defined as a crop sequence with the identified crop appearing at
## least 3 times during the reference period
# Specilization function uses the crop image, reference period (as list), crop CDL code, crop name (for bands), and critical value to measure specialization (val)
def specialization(img, listRef, code, cropname, val):
  sp= img.select(listRef).eq(code)
  z= sp.expression(
      '(A+B+C+D+E+F+G)',
      {'A': sp.select(0),
        'B': sp.select(1),
        'C': sp.select(2),
        'D': sp.select(3),
        'E': sp.select(4),
        'F': sp.select(5),
        'G': sp.select(6)
        })
  spec= z.gte(val)
  new_band = ee.Image(spec).rename(cropname+'_specialized')
  return new_band

In [ ]:
# Specialized areas period 1
listRef_years= np.arange(2008,2015,1)
listRef= [str(year) + '_remapped' for year in listRef_years]

corn_spe= specialization(corn_coll, listRef, 1, 'corn', 3)
soy_spe= specialization(soy_coll, listRef, 5, 'soy', 3)

cot_spe= specialization(cot_coll, listRef, 2, 'cotton', 3)

rice_spe= specialization(rice_coll, listRef, 3, 'rice', 3)

In [ ]:
# Specialized areas period 2

listRef2_years = np.arange(2013,2020,1)
listRef2 = [str(year) + '_remapped' for year in listRef2_years]

corn_spe2= specialization(corn_coll, listRef2, 1, 'corn', 3)
soy_spe2= specialization(soy_coll, listRef2, 5, 'soy', 3)

cot_spe2= specialization(cot_coll, listRef2, 2, 'cotton', 3)

rice_spe2= specialization(rice_coll, listRef2, 3, 'rice', 3)

In [ ]:
soy_spe2.getInfo()

{'type': 'Image',
 'bands': [{'id': 'soy_specialized',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1},
   'dimensions': [153811, 96523],
   'crs': 'PROJCS["Albers Conical Equal Area", \n  GEOGCS["NAD83", \n    DATUM["North_American_Datum_1983", \n      SPHEROID["GRS 1980", 6378137.0, 298.2572221010002, AUTHORITY["EPSG","7019"]], \n      AUTHORITY["EPSG","6269"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH], \n    AUTHORITY["EPSG","4269"]], \n  PROJECTION["Albers_Conic_Equal_Area"], \n  PARAMETER["central_meridian", -96.0], \n  PARAMETER["latitude_of_origin", 23.0], \n  PARAMETER["standard_parallel_1", 29.5], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["standard_parallel_2", 45.5], \n  UNIT["m", 1.0], \n  AXIS["x", EAST], \n  AXIS["y", NORTH]]',
   'crs_transform': [30, 0, -2356095, 0, -30, 3172605]}]}

## **Identify non specialized cropland**

In [ ]:
## Function to identified cropland that never had the given commodity crop during the reference period
def nonspecialization(img, filter_img, listRef, code, cropname):
  sp= img.select(listRef).eq(code)
  z= sp.expression(
      '(A+B+C+D+E+F+G)',
      {'A': sp.select(0),
        'B': sp.select(1),
        'C': sp.select(2),
        'D': sp.select(3),
        'E': sp.select(4),
        'F': sp.select(5),
        'G': sp.select(6)
        })
  nonspec= z.eq(0)

  net= nonspec.multiply(filter_img) ## remove land that is always fallow
  new_band = ee.Image(net).rename(cropname+'_nonspecialized')
  return new_band

In [ ]:
# Non-specialized areas period 1

listRef_years= np.arange(2008,2015,1)
listRef= [str(year) + '_remapped' for year in listRef_years]

corn_nospe= nonspecialization(corn_coll, nonfallow_img, listRef, 1, 'corn')
soy_nospe= nonspecialization(soy_coll, nonfallow_img,  listRef, 5, 'soy')
cot_nospe= nonspecialization(cot_coll, nonfallow_img,  listRef, 2, 'cotton')
rice_nospe= nonspecialization(rice_coll, nonfallow_img, listRef, 3, 'rice')

NameError: name 'nonfallow_img' is not defined

In [ ]:
# Non-specialized areas period 2

#listRef2_years = np.arange(2013,2020,1)
#listRef2 = [str(year) + '_remapped' for year in listRef2_years]

corn_nospe2= nonspecialization(corn_coll, nonfallow_img2, listRef2, 1, 'corn')
soy_nospe2= nonspecialization(soy_coll, nonfallow_img2,  listRef2, 5, 'soy')
cot_nospe2= nonspecialization(cot_coll, nonfallow_img2,  listRef2, 2, 'cotton')
rice_nospe2= nonspecialization(rice_coll, nonfallow_img2, listRef2, 3, 'rice')

##**SWITCHING AWAY: Funtion to identify cropswitches FROM specialized crop sequences**

In [ ]:
#def cropswitching_specialized_areas(img, img_masked, img_onlycrops, comm_img, listRef, listAssess, label):
def cropswitching_specialized_areas(img, img_masked, img_onlycrops, listRef, listAssess, label):
  ref_img= img.select(listRef) # Reference Image years
  assess_img= img.select(list(range(listRef[-1]+1,listAssess[-1]+1)))  # Assessment years in band number
  cs_img= ee.Image() ## empty image for commodity crop switches
  ## For each assessment year calculate cropswitching property

  for n in listAssess:
    cs= ref_img.eq(img.select(str(n)+'_remapped')) # comparing years
    x= cs.expression(
    '(A+B+C+D+E+F+G)',
      {'A': cs.select(0),
        'B': cs.select(1),
        'C': cs.select(2),
        'D': cs.select(3),
        'E': cs.select(4),
        'F': cs.select(5),
        'G': cs.select(6)
        })
    css= x.eq(0)
    crop= css.multiply(img_masked) #multiply by crop-specialized cells (1,0) one band
    cropsw= crop.multiply(img_onlycrops.select(str(n)+'_cropland')) #multiply by only appropriate crops (remove fallow, other crops, alfalfa)
    #comm= crop.multiply(comm_img.select(str(n)+'_cropland')) # identify switches into Commodity crops
    new_band = ee.Image(cropsw).rename(str(n)+label) #new band with binary crop switches
    #new_bandc = ee.Image(comm).rename(str(n)+label) #new band with Commodity crop switches
    cs_img= cs_img.addBands(new_band) ## Image with crop switches per year
    #assess_imgc= assess_imgc.addBands(new_bandc) ##Image with Commodity crop switches
  return  cs_img #, assess_imgc

In [ ]:
n=7
listRef= np.arange(2008,2015,1)
listAssess= np.arange(2015,2020,1)


7

In [ ]:
# Specific crop switches
cs_fromcorn= ee.Image()
cs_fromcorn, cs_fromcorn_comm= cropswitching_specialized_areas(corn_coll, corn_spe, mask_img, commodity_coll, listRef, listAssess, '_fromcorn')
cs_fromcorn= cs_fromcorn.select(list(range(1+ len(listRef),1+ len(listRef)+len(listAssess)))) #remove the constant term and keep only annual switching cells
cs_fromcorn_comm= cs_fromcorn_comm.select(list(range(1,1+len(listAssess))))

cs_fromsoy= ee.Image()
cs_fromsoy, cs_fromsoy_comm = cropswitching_specialized_areas(soy_coll, soy_spe, mask_img,commodity_coll, listRef, listAssess, '_fromsoy')
cs_fromsoy= cs_fromsoy.select(list(range(1+ len(listRef),1+ len(listRef)+len(listAssess))))
cs_fromsoy_comm= cs_fromsoy_comm.select(list(range(1,1+len(listAssess))))

cs_fromcot= ee.Image()
cs_fromcot, cs_fromcot_comm= cropswitching_specialized_areas(cot_coll, cot_spe, mask_img, commodity_coll, listRef, listAssess, '_fromcot')
cs_fromcot= cs_fromcot.select(list(range(1+ len(listRef),1+ len(listRef)+len(listAssess))))
cs_fromcot_comm= cs_fromcot_comm.select(list(range(1,1+len(listAssess))))

cs_fromrice= ee.Image()
cs_fromrice, cs_fromrice_comm= cropswitching_specialized_areas(rice_coll, rice_spe, mask_img, commodity_coll, listRef, listAssess, '_fromrice')
cs_fromrice= cs_fromrice.select(list(range(1+ len(listRef),1+ len(listRef)+len(listAssess))))
cs_fromrice_comm= cs_fromrice_comm.select(list(range(1,1+len(listAssess))))

In [ ]:
## Total cropswitches

cs_fromcorn= ee.Image()
cs_fromcorn = cropswitching_specialized_areas(corn_coll, corn_spe, mask_img, listRef, listAssess, '_fromcorn')
cs_fromcorn= cs_fromcorn.select(list(range(1,1+ len(listAssess)))) #remove the constant term and keep only annual switching cells

cs_fromsoy= ee.Image()
cs_fromsoy = cropswitching_specialized_areas(soy_coll, soy_spe, mask_img, listRef, listAssess, '_fromsoy')
cs_fromsoy= cs_fromsoy.select(list(range(1,1+ len(listAssess))))

cs_fromcot= ee.Image()
cs_fromcot= cropswitching_specialized_areas(cot_coll, cot_spe, mask_img listRef, listAssess, '_fromcot')
cs_fromcot= cs_fromcot.select(list(range(1,1+ len(listAssess))))

cs_fromrice= ee.Image()
cs_fromrice = cropswitching_specialized_areas(rice_coll, rice_spe, mask_img, listRef, listAssess, '_fromrice')
cs_fromrice= cs_fromrice.select(list(range(1,1+ len(listAssess))))

## **Crop switching AWAY FROM coommodity crops between reference period 2 (2013-2019) and assessment period 2 (2020-2023)**

In [ ]:

listRef2= np.arange(2013,2020,1)
listAssess2= np.arange(2020,2024,1)

In [ ]:
cs_fromcorn2= ee.Image()
cs_fromcorn2 = cropswitching_specialized_areas(corn_coll, corn_spe, mask_img, listRef2, listAssess2, '_fromcorn')
cs_fromcorn2= cs_fromcorn2.select(list(range(2,2+ len(listAssess2)))) #remove the constant term and keep only annual switching cells

cs_fromsoy2= ee.Image()
cs_fromsoy2 = cropswitching_specialized_areas(soy_coll, soy_spe, mask_img, listRef2, listAssess2, '_fromsoy')
cs_fromsoy2= cs_fromsoy2.select(list(range(2, 2+ len(listAssess2))))

cs_fromcot2= ee.Image()
cs_fromcot2= cropswitching_specialized_areas(cot_coll, cot_spe, mask_img, listRef2, listAssess2, '_fromcot')
cs_fromcot2= cs_fromcot2.select(list(range(2, 2 + len(listAssess2))))

cs_fromrice2= ee.Image()
cs_fromrice2 = cropswitching_specialized_areas(rice_coll, rice_spe, mask_img, listRef2, listAssess2, '_fromrice')
cs_fromrice2= cs_fromrice2.select(list(range(2, 2 + len(listAssess2))))

EEException: Invalid argument specified for ee.List(): [2013 2014 2015 2016 2017 2018 2019]



#**SWITCHING INTO: Function to identify crop switches INTO commodity crops**

In [ ]:
## Identified switches from non-specialized sequences to specific crop types (1= cropswitch to given crop type)

def cropswitch_variety(img, filter_img, listRef, listAssess, code, label):
  ref_img= img.select(listRef) # Reference Image years
  assess_img= img.select(list(range(listRef[-1]+1,listAssess[-1])+1))  # Assessment years in years
  cs_img= ee.Image() ## empty image for crop switches
  ##
  ## For each assessment year calculate cropswitching property

  for n in listAssess:
    cs= ref_img.eq(img.select(str(n)+'_remapped')) # comparing years
    x= cs.expression(
    '(A+B+C+D+E+F+G)',
      {'A': cs.select(0),
       'B': cs.select(1),
       'C': cs.select(2),
       'D': cs.select(3),
       'E': cs.select(4),
       'F': cs.select(5),
       'G': cs.select(6)
      })
    css= x.eq(0)
    ## multiply by crop codes and then get only crops with code###
    cropsw= css.multiply(img.select(str(n)+'_remapped')) #multiply by crop codes
    cscode= cropsw.eq(code) #select switches with crop code
    csf= cscode.multiply(filter_img) #multiply by nonspecialized crop x land
    new_band = ee.Image(csf).rename(str(n)+ label) # Add new band: label= year from listAssess + personalized label
    cs_img= cs_img.addBands(new_band)

  return  cs_img     # Return cropswitching for assessment periods


In [ ]:
# THIS IS AGGREGATED (sum) and DOWNLOADED (cropswitch columns)
cs_tocorn= ee.Image()
cs_tocorn= cropswitch_variety(corn_coll, corn_nospe, listRef, listAssess, 1, '_tocorn')
cs_tocorn= cs_tocorn.select(list(range(2,2+ len(listAssess)))) #remove the constant term and keep only cropswitch columns

cs_tosoy= ee.Image()
cs_tosoy= cropswitch_variety(soy_coll, soy_nospe, listRef, listAssess, 5, '_tosoy')
cs_tosoy= cs_tosoy.select(list(range(2,2+ len(listAssess))))

cs_tocot= ee.Image()
cs_tocot= cropswitch_variety(cot_coll, cot_nospe,  listRef, listAssess, 2, '_tocot')
cs_tocot= cs_tocot.select(list(range(2,2+ len(listAssess))))

cs_torice= ee.Image()
cs_torice= cropswitch_variety(rice_coll, rice_nospe,  listRef, listAssess, 3, '_torice')
cs_torice= cs_torice.select(list(range(2,2+ len(listAssess))))

In [ ]:
## Period 2
listRef2= np.arange(2013,2020,1)
listAssess2= np.arange(2020,2024,1)

cs_tocorn2= ee.Image()
cs_tocorn2= cropswitch_variety(corn_coll, corn_nospe, listRef2, listAssess2, 1, '_tocorn')
cs_tocorn2= cs_tocorn2.select(list(range(2,2+ len(listAssess2)))) #remove the constant term and keep only cropswitch columns

cs_tosoy2= ee.Image()
cs_tosoy2= cropswitch_variety(soy_coll, soy_nospe, listRef2, listAssess2, 5, '_tosoy')
cs_tosoy2= cs_tosoy2.select(list(range(2,2+ len(listAssess2))))

cs_tocot2= ee.Image()
cs_tocot2= cropswitch_variety(cot_coll, cot_nospe,  listRef, listAssess, 2, '_tocot')
cs_tocot2= cs_tocot2.select(list(range(2,2+ len(listAssess))))

cs_torice2= ee.Image()
cs_torice2= cropswitch_variety(rice_coll, rice_nospe,  listRef2, listAssess2, 3, '_torice')
cs_torice2= cs_torice2.select(list(range(2,2+ len(listAssess2))))

In [ ]:
#cs_torice.getInfo()

In [ ]:
def land_percrop(img, listAssess, code):

  new_img= ee.Image()

  ## For each assessment year calculate cropswitching property

  for n in listAssess:
    g= img.select(str(n)+'_remapped').eq(code) # comparing years
    gn = ee.Image(g).rename(str(n)+'_cropland')
    new_img= new_img.addBands(gn)

  return  new_img

In [ ]:
listAssess= np.arange(2014,2022,1)
corn_land= land_percrop(corn_coll, listAssess, 1)
soy_land= land_percrop(soy_coll, listAssess, 5)
sorg_land= land_percrop(sorg_coll, listAssess, 4)
cot_land= land_percrop(cot_coll, listAssess, 2)
bar_land= land_percrop(bar_coll, listAssess, 21)
wheat_land= land_percrop(wheat_coll, listAssess, 22)
rice_land= land_percrop(rice_coll, listAssess, 3)

#**Reduce pixels to county aggregates**

In [ ]:
feature_coll = ee.FeatureCollection('TIGER/2018/Counties')

In [ ]:
# Small test
county = feature_coll.filter(ee.Filter.Or(ee.Filter.eq('GEOID', '06001'),ee.Filter.eq('GEOID', '56023')))
reducer = ee.Reducer.sum().unweighted()

# Reduce regions
reduced_total = img_temp.select(0).reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)

#print(reduced_fc.first().getInfo())

In [ ]:
### TESTING with 2 counties
county = feature_coll.filter(ee.Filter.Or(ee.Filter.eq('GEOID', '22117'),ee.Filter.eq('GEOID', '56009')))
reducer = ee.Reducer.sum().unweighted()

# Reduce regions
reduced_total = nonfallow_img.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


features = reduced_total.getInfo()['features']


# Extract the properties from each feature
data = [feature['properties'] for feature in features]

df = pd.DataFrame(data)

In [ ]:
print('specialized: ', df1)
print('nonspecialized: ', df2) # sum
print('cropswitch to: ', df3)
print('cropswitch from: ', df4)
print('total cropland area', df) # count non specialized

In [ ]:
df # non fallow

,ALAND,AWATER,CBSAFP,CLASSFP,COUNTYFP,COUNTYNS,CSAFP,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,METDIVFP,MTFCC,NAME,NAMELSAD,STATEFP,sum
0,1734074152,16457670,14220,H1,117,00559557,406,A,22117,+30.8521435,-090.0462534,15,,G4020,Washington,Washington Parish,22,0
1,11020303965,26150335,,H1,009,01605070,,A,56009,+42.9846234,-105.5247521,06,,G4020,Converse,Converse County,56,36296


In [ ]:
df1 # corn sum specialized

,ALAND,AWATER,CBSAFP,CLASSFP,COUNTYFP,COUNTYNS,CSAFP,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,METDIVFP,MTFCC,NAME,NAMELSAD,STATEFP,sum
0,1734074152,16457670,14220,H1,117,00559557,406,A,22117,+30.8521435,-090.0462534,15,,G4020,Washington,Washington Parish,22,0
1,11020303965,26150335,,H1,009,01605070,,A,56009,+42.9846234,-105.5247521,06,,G4020,Converse,Converse County,56,1240


In [ ]:
df2 # corn count

,ALAND,AWATER,CBSAFP,CLASSFP,COUNTYFP,COUNTYNS,CSAFP,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,METDIVFP,MTFCC,NAME,NAMELSAD,STATEFP,count
0,1734074152,16457670,14220,H1,117,00559557,406,A,22117,+30.8521435,-090.0462534,15,,G4020,Washington,Washington Parish,22,0
1,11020303965,26150335,,H1,009,01605070,,A,56009,+42.9846234,-105.5247521,06,,G4020,Converse,Converse County,56,36305


In [ ]:
df3 # corn non spec

,ALAND,AWATER,CBSAFP,CLASSFP,COUNTYFP,COUNTYNS,CSAFP,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,METDIVFP,MTFCC,NAME,NAMELSAD,STATEFP,sum
0,1734074152,16457670,14220,H1,117,00559557,406,A,22117,+30.8521435,-090.0462534,15,,G4020,Washington,Washington Parish,22,0
1,11020303965,26150335,,H1,009,01605070,,A,56009,+42.9846234,-105.5247521,06,,G4020,Converse,Converse County,56,27112


In [ ]:
df4 # cs from corn spe

,2015_fromcorn,2016_fromcorn,2017_fromcorn,2018_fromcorn,2019_fromcorn,2020_fromcorn,2021_fromcorn,2022_fromcorn,ALAND,AWATER,...,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,METDIVFP,MTFCC,NAME,NAMELSAD,STATEFP
0,0,0,0,0,0,0,0,0,1734074152,16457670,...,A,22117,+30.8521435,-090.0462534,15,,G4020,Washington,Washington Parish,22
1,49,108,12,9,17,33,18,25,11020303965,26150335,...,A,56009,+42.9846234,-105.5247521,06,,G4020,Converse,Converse County,56


In [ ]:
df5 # cs to corn non spe

,2015_tocorn,2016_tocorn,2017_tocorn,2018_tocorn,2019_tocorn,2020_tocorn,2021_tocorn,2022_tocorn,ALAND,AWATER,...,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,METDIVFP,MTFCC,NAME,NAMELSAD,STATEFP
0,0,0,0,0,0,0,0,0,1734074152,16457670,...,A,22117,+30.8521435,-090.0462534,15,,G4020,Washington,Washington Parish,22
1,57,478,433,1339,803,1200,1307,1106,11020303965,26150335,...,A,56009,+42.9846234,-105.5247521,06,,G4020,Converse,Converse County,56


In [ ]:
df

,ALAND,AWATER,CBSAFP,CLASSFP,COUNTYFP,COUNTYNS,CSAFP,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,METDIVFP,MTFCC,NAME,NAMELSAD,STATEFP,sum
0,4117546676,1133055836,19300,H1,003,00161527,380,A,01003,+30.6592183,-087.7460666,06,,G4020,Baldwin,Baldwin County,01,2360
1,11020303965,26150335,,H1,009,01605070,,A,56009,+42.9846234,-105.5247521,06,,G4020,Converse,Converse County,56,82


**Selected Counties in the contiguous US**

In [ ]:
# select counties in the continuous US
county = feature_coll.filter(ee.Filter.Or(ee.Filter.eq('STATEFP', '13'), ee.Filter.eq('STATEFP', '16'), ee.Filter.eq('STATEFP', '17'), ee.Filter.eq('STATEFP', '18'),
           ee.Filter.eq('STATEFP', '19'), ee.Filter.eq('STATEFP', '20'), ee.Filter.eq('STATEFP', '10'), ee.Filter.eq('STATEFP', '12'), ee.Filter.eq('STATEFP', '09'),
           ee.Filter.eq('STATEFP', '21'), ee.Filter.eq('STATEFP', '22'), ee.Filter.eq('STATEFP', '23'), ee.Filter.eq('STATEFP', '24'), ee.Filter.eq('STATEFP', '25'),
           ee.Filter.eq('STATEFP', '26'),
           ee.Filter.eq('STATEFP', '27'), ee.Filter.eq('STATEFP', '28'), ee.Filter.eq('STATEFP', '29'), ee.Filter.eq('STATEFP', '30'), ee.Filter.eq('STATEFP', '31'),
           ee.Filter.eq('STATEFP', '32'), ee.Filter.eq('STATEFP', '33'), ee.Filter.eq('STATEFP', '34'), ee.Filter.eq('STATEFP', '35'), ee.Filter.eq('STATEFP', '36'),
           ee.Filter.eq('STATEFP', '37'), ee.Filter.eq('STATEFP', '38'), ee.Filter.eq('STATEFP', '39'), ee.Filter.eq('STATEFP', '40'), ee.Filter.eq('STATEFP', '41'),
           ee.Filter.eq('STATEFP', '42'), ee.Filter.eq('STATEFP', '44'), ee.Filter.eq('STATEFP', '45'), ee.Filter.eq('STATEFP', '46'), ee.Filter.eq('STATEFP', '47'),
           ee.Filter.eq('STATEFP', '48'), ee.Filter.eq('STATEFP', '49'), ee.Filter.eq('STATEFP', '50'), ee.Filter.eq('STATEFP', '51'), ee.Filter.eq('STATEFP', '53'),
           ee.Filter.eq('STATEFP', '54'), ee.Filter.eq('STATEFP', '55'), ee.Filter.eq('STATEFP', '56'), ee.Filter.eq('STATEFP', '01'), ee.Filter.eq('STATEFP', '04'),
           ee.Filter.eq('STATEFP', '05'), ee.Filter.eq('STATEFP', '06'), ee.Filter.eq('STATEFP', '08')))



In [ ]:
########### CORN LAND
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = corn_land.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='corn_land',
        folder='Farmers adaptation',
        fileNamePrefix='corn_land',
        fileFormat='CSV')

task.start()

In [ ]:
########### SOY LAND
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = soy_land.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='soy_land',
        folder='Farmers adaptation',
        fileNamePrefix='soy_land',
        fileFormat='CSV')

task.start()

In [ ]:
########### COTTON LAND
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cot_land.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='cotton_land',
        folder='Farmers adaptation',
        fileNamePrefix='cotton_land',
        fileFormat='CSV')

task.start()

In [ ]:
########### RICE LAND
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = rice_land.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='rice_land',
        folder='Farmers adaptation',
        fileNamePrefix='rice_land',
        fileFormat='CSV')

task.start()

##**Specialized cells**

In [ ]:
########### Specialized cells in CORN
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = corn_spe.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='corn_specialized',
        folder='Farmers adaptation',
        fileNamePrefix='corn_specialized',
        fileFormat='CSV')

task.start()

In [ ]:
########### Specialized cells in SOY
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = soy_spe.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='soy_specialized',
        folder='Farmers adaptation',
        fileNamePrefix='soy_specialized',
        fileFormat='CSV')

task.start()

In [ ]:
########### Specialized cells in COTTON
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cot_spe.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='cotton_specialized',
        folder='Farmers adaptation',
        fileNamePrefix='cotton_specialized',
        fileFormat='CSV')

task.start()

In [ ]:
########### Specialized cells in RICE
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = rice_spe.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='rice_specialized',
        folder='Farmers adaptation',
        fileNamePrefix='rice_specialized',
        fileFormat='CSV')

task.start()

##**Specialized cells period 2**

In [ ]:
########### Specialized cells in CORN
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = corn_spe2.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='corn_specialized2',
        folder='Farmers adaptation',
        fileNamePrefix='corn_specialized2',
        fileFormat='CSV')

task.start()

In [ ]:
########### Specialized cells in SOY
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = soy_spe2.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='soy_specialized2',
        folder='Farmers adaptation',
        fileNamePrefix='soy_specialized2',
        fileFormat='CSV')

task.start()

In [ ]:
########### Specialized cells in COTTON
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cot_spe2.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='cotton_specialized2',
        folder='Farmers adaptation',
        fileNamePrefix='cotton_specialized2',
        fileFormat='CSV')

task.start()

In [ ]:
########### Specialized cells in RICE
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = rice_spe2.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='rice_specialized2',
        folder='Farmers adaptation',
        fileNamePrefix='rice_specialized2',
        fileFormat='CSV')

task.start()

##**Non specialized cells**

In [ ]:
########### cells WITHOUT CORN
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = corn_nospe.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='corn_inexistent',
        folder='Farmers adaptation',
        fileNamePrefix='corn_inexistent',
        fileFormat='CSV')

task.start()

In [ ]:
########### cells WITHOUT SOY
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = soy_nospe.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='soy_inexistent',
        folder='Farmers adaptation',
        fileNamePrefix='soy_inexistent',
        fileFormat='CSV')

task.start()

In [ ]:
########### cells WITHOUT COTTON
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cot_nospe.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='cotton_inexistent',
        folder='Farmers adaptation',
        fileNamePrefix='cotton_inexistent',
        fileFormat='CSV')

task.start()

In [ ]:
########### cells WITHOUT RICE
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = rice_nospe.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='rice_inexistent',
        folder='Farmers adaptation',
        fileNamePrefix='rice_inexistent',
        fileFormat='CSV')

task.start()

##**Non Specialized cells Period 2**

In [ ]:
########### cells WITHOUT CORN
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = corn_nospe2.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='corn_inexistent2',
        folder='Farmers adaptation',
        fileNamePrefix='corn_inexistent2',
        fileFormat='CSV')

task.start()

In [ ]:
########### cells WITHOUT SOY
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = soy_nospe2.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='soy_inexistent2',
        folder='Farmers adaptation',
        fileNamePrefix='soy_inexistent2',
        fileFormat='CSV')

task.start()

In [ ]:
########### cells WITHOUT COTTON
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cot_nospe2.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='cotton_inexistent2',
        folder='Farmers adaptation',
        fileNamePrefix='cotton_inexistent2',
        fileFormat='CSV')

task.start()

In [ ]:
########### cells WITHOUT RICE
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = rice_nospe2.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='rice_inexistent2',
        folder='Farmers adaptation',
        fileNamePrefix='rice_inexistent2',
        fileFormat='CSV')

task.start()

#**Total cropland area (specialized and non specialized)**

In [ ]:
########### CROPLAND cells valid throughout the reference period USING NONFALLOW Image
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = nonfallow_img.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='onlycropland',
        folder='Farmers adaptation',
        fileNamePrefix='onlycropland',
        fileFormat='CSV')

task.start()

##**Crop switches (FROM specialized Commodity crops) period 1**

In [ ]:
########### Switches FROM CORN
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_fromcorn.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_fromcorn',
        folder='Farmers adaptation',
        fileNamePrefix='switches_fromcorn',
        fileFormat='CSV')

task.start()

In [ ]:
########### Switches FROM SOY
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_fromsoy.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_fromsoy',
        folder='Farmers adaptation',
        fileNamePrefix='switches_fromsoy',
        fileFormat='CSV')

task.start()

In [ ]:
########### Switches FROM COTTON
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_fromcot.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_fromcotton',
        folder='Farmers adaptation',
        fileNamePrefix='switches_fromcotton',
        fileFormat='CSV')

task.start()

In [ ]:
########### Switches FROM RICE
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_fromrice.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_fromrice',
        folder='Farmers adaptation',
        fileNamePrefix='switches_fromrice',
        fileFormat='CSV')

task.start()

#**Commodity Crop switches (FROM Commodity-specialized crop sequences)**

In [ ]:
########### Commodity Switches FROM CORN
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_fromcorn_comm.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,
        description='switches_fromcorn_commodity',
        folder='Farmers adaptation',
        fileNamePrefix='switches_fromcorn_comm',
        fileFormat='CSV')

task.start()

In [ ]:
########### Commodity Switches FROM SOY
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_fromsoy_comm.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,
        description='switches_fromsoy_commodity',
        folder='Farmers adaptation',
        fileNamePrefix='switches_fromsoy_comm',
        fileFormat='CSV')

task.start()

In [ ]:
########### Commodity Switches FROM COTTON
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_fromcot_comm.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_fromcotton_commodity',
        folder='Farmers adaptation',
        fileNamePrefix='switches_fromcotton_comm',
        fileFormat='CSV')

task.start()

In [ ]:
########### Commodity Switches FROM RICE
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_fromrice_comm.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_fromrice_commodity',
        folder='Farmers adaptation',
        fileNamePrefix='switches_fromrice_comm',
        fileFormat='CSV')

task.start()

## **Crop switches FROM commodity-specialized crop sequences Period 2**

In [ ]:
########### Switches FROM CORN
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_fromcorn2.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_fromcorn2',
        folder='Farmers adaptation',
        fileNamePrefix='switches_fromcorn2',
        fileFormat='CSV')

task.start()

In [ ]:
########### Switches FROM SOY
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_fromsoy2.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_fromsoy2',
        folder='Farmers adaptation',
        fileNamePrefix='switches_fromsoy2',
        fileFormat='CSV')

task.start()

In [ ]:
########### Switches FROM COTTON
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_fromcot2.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_fromcotton2',
        folder='Farmers adaptation',
        fileNamePrefix='switches_fromcotton2',
        fileFormat='CSV')

task.start()

In [ ]:
########### Switches FROM RICE
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_fromrice2.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_fromrice2',
        folder='Farmers adaptation',
        fileNamePrefix='switches_fromrice2',
        fileFormat='CSV')

task.start()

#**Crop Switches INTO Commodity crops**

In [ ]:
########### Switches TO CORN
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_tocorn.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_tocorn',
        folder='Farmers adaptation',
        fileNamePrefix='switches_tocorn',
        fileFormat='CSV')

task.start()

In [ ]:
########### Switches TO SOY
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_tosoy.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_tosoy',
        folder='Farmers adaptation',
        fileNamePrefix='switches_tosoy',
        fileFormat='CSV')

task.start()

In [ ]:
########### Switches TO COTTON
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_tocot.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_tocotton',
        folder='Farmers adaptation',
        fileNamePrefix='switches_tocotton',
        fileFormat='CSV')

task.start()

In [ ]:
########### Switches TO RICE
reducer = ee.Reducer.sum().unweighted()
#
# Reduce regions
reduced_total = cs_torice.reduceRegions(
    collection= county,
    reducer=reducer,
    scale=30,  # Pixel scale in meters
    tileScale=16)


#print(reduced_fc.first().getInfo())
name= ''
task = ee.batch.Export.table.toDrive(
        collection=reduced_total,  # Use 'collection' instead of 'image'
        description='switches_torice',
        folder='Farmers adaptation',
        fileNamePrefix='switches_torice',
        fileFormat='CSV')

task.start()